In [1]:
# load dependencies

import os

import yaml
import json
import re

import process
from sklearn.feature_extraction.text import TfidfVectorizer

from importlib import reload
reload(process)

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pickle

from dotenv import load_dotenv
load_dotenv();

In [7]:
# define working directories
home = os.getenv("PROJ_HOME")
modeling_dir = os.path.join(home, "explorer_ai/modeling")
rasa_dir = os.path.join(home, "explorer_ai/rasa")
intents_import_filepath = os.path.join(rasa_dir, "data/nlu/intents.yml")

In [18]:
# load intents from rasa training data
with open(intents_import_filepath, "r") as yml_file:
    intents = yaml.safe_load(yml_file)

In [19]:
# regex pattern for entity annotation
entity_pattern = re.compile(r"\[(.+?)\]\((.+?)\)")

# chatbot responses
utter_try_something_else = '''Sorry, but I don't know how to respond to that. Try something like these instead:
  * I'd like some information about electromagnetic radiation.
  * What is information theory?
  * Please tell me about the geologic history of Earth.'''
utterances = {
    "greet": "Hello there!",
    "thanks": "Glad I could be of assistance :)",
    "goodbye": "Bye-bye!",
    "bot_challenge": "I'm Explorer AI, an educational chatbot.",
    "out_of_scope": utter_try_something_else,
    "ask_for_explanation": "You'd like to get an explanation of something, is that correct?",
    "stop": "Are you sure you'd like to stop?",
    "affirm": "Noted!",
    "deny": "Got it!",
}

sentences = []
labels = []
all_labels = []
responses = []
# entities = []
data = []

for item in intents["nlu"]:
    label = item["intent"]
    this_item = {
        "intent": label,
        "examples": [],
        "responses": utterances[label]
    }
    for example in item["examples"]:
        entity_match = re.search(entity_pattern, example)
        if entity_match:
            entity_text = entity_match.group(1)
            entity_type = entity_match.group(2)
            # entities.append(tuple((entity_text, entity_type)))
            example = example.replace(entity_match.group(0), entity_text)
        this_item["examples"].append(example)
        proc_example = process.preprocess(example,
                                          stop_words=False,
                                          punctuation=True,
                                          remove_num=True)
        sentences.append(proc_example)
        labels.append(label)
        if label not in all_labels:
            all_labels.append(label)
    data.append(this_item)

        # if label == "greet":
        #     responses.append(utterances["greet"])
        # elif label == "goodbye":
        #     responses.append(utterances["goodbye"])
        # elif label == "thanks":
        #     responses.append(utterances["thanks"])
        # elif label == "ask_for_explanation":
        #     responses.append(utterances["ask_for_explanation"])
        # elif label == "stop":
        #     responses.append(utterances["stop"])
        # elif label == "affirm":
        #     responses.append(utterances["affirm"])
        # elif label == "deny":
        #     responses.append(utterances["deny"])
        # elif label == "bot_challenge":
        #     responses.append(utterances["bot_challenge"])
        # elif label == "out_of_scope":
        #     responses.append(utterances["out_of_scope"])

num_classes = len(all_labels)

In [20]:
# save intents, examples, responses to json file for chatbot use
# Q: is "examples" key needed?

os.remove("intents.json")

with open("intents.json", "w") as out_file:
    json.dump(data, out_file, indent=2)

In [21]:
print("{} sentences".format(len(sentences)))
print("{} labels".format(num_classes))

189 sentences
9 labels


In [22]:
# encode labels

encoder = LabelEncoder()
encoder.fit(labels)
labels = encoder.transform(labels)

In [23]:
# model parameters

vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, truncating="post", maxlen=max_len)

In [24]:
# model architecture

model = Sequential()

model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(num_classes, activation="softmax"))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 9)                 153       
                                                                 
Total params: 16,697
Trainable params: 16,697
Non-trainable params: 0
____________________________________________________

In [25]:
# train model

epochs = 500
history = model.fit(padded_sequences, np.array(labels), epochs=epochs)

Epoch 1/500


2023-01-25 14:56:49.799578: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


6/6 [==============================] - 0s 705us/step - loss: 2.1948 - accuracy: 0.1958
Epoch 2/500
6/6 [==============================] - 0s 693us/step - loss: 2.1897 - accuracy: 0.2116
Epoch 3/500
6/6 [==============================] - 0s 672us/step - loss: 2.1852 - accuracy: 0.2116
Epoch 4/500
6/6 [==============================] - 0s 645us/step - loss: 2.1803 - accuracy: 0.2116
Epoch 5/500
6/6 [==============================] - 0s 697us/step - loss: 2.1753 - accuracy: 0.2116
Epoch 6/500
6/6 [==============================] - 0s 774us/step - loss: 2.1690 - accuracy: 0.2116
Epoch 7/500
6/6 [==============================] - 0s 729us/step - loss: 2.1642 - accuracy: 0.2116
Epoch 8/500
6/6 [==============================] - 0s 755us/step - loss: 2.1575 - accuracy: 0.2116
Epoch 9/500
6/6 [==============================] - 0s 694us/step - loss: 2.1507 - accuracy: 0.2116
Epoch 10/500
6/6 [==============================] - 0s 709us/step - loss: 2.1445 - accuracy: 0.2116
Epoch 11/500
6/6 [===

In [26]:
# save model, tokenizer, label encoder

model.save("chat_model")

with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open("label_encoder.pickle", "wb") as ecn_file:
    pickle.dump(encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

2023-01-25 14:56:57.861450: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: chat_model/assets
